In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
import pandas as pd
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tqdm import tqdm

import pickle

from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
zh_wiki_id = open("data/zh_wiki_id_w_d").readline()
word_to_id = pickle.load(open("data/word_to_id_w_d.pkl", "rb"))
id_to_word = pickle.load(open("data/id_to_word_w_d.pkl", "rb"))
# word_count = pickle.load(open("data/count.pkl", "rb"))

In [4]:
len(id_to_word), len(word_to_id)#, len(word_count)

(507260, 507260)

(504175, 504175, 504175)

In [5]:
def getWord(data, num, data_index):
    sub_data_string = data[data_index:data_index+num*(6+1)]
    result = []
    for index, item in enumerate(sub_data_string.split()):
        if index == num: break
        data_index += len(item) + 1
        result.append(int(item))
    if len(result) < num:
        return getWord(data, num, 0)
    assert len(result) == num
    return result, data_index

In [6]:
def generate_batch(batch_size, skip_window, num_skips):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size, num_skips), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    assert batch_size >= span
    buffer = collections.deque(maxlen=span)  # pylint: disable=redefined-builti
    
    result, data_index = getWord(zh_wiki_id, span, data_index)
    buffer.extend(result)
    
    for i in range(batch_size):
        context_words = [w for w in range(span) if w != skip_window]
        batch[i, :] = [buffer[token] for idx, token in enumerate(context_words)]
        labels[i, 0] = buffer[skip_window]
        result, data_index = getWord(zh_wiki_id, 1, data_index)
        buffer.append(result[0])
        if data_index > len(zh_wiki_id):
            result, data_index = getWord(zh_wiki_id, span-1, 0)
            buffer.extend(result)
        if i == batch_size - span:
            last_index = data_index
            
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = last_index
    return batch, labels

In [7]:
data_index = 0

batch, labels = generate_batch(batch_size=8, skip_window=1, num_skips=2*1)
for i in range(8):
    print(batch[i, 0], id_to_word[batch[i, 0]],
          batch[i, 1], id_to_word[batch[i, 1]],
          '->', labels[i, 0], id_to_word[labels[i, 0]])

1348 数学 501 利用 -> 9 是
9 是 237319 符号语言 -> 501 利用
501 利用 141 研究 -> 237319 符号语言
237319 符号语言 894 数量 -> 141 研究
141 研究 5 、 -> 894 数量
894 数量 499 结构 -> 5 、
5 、 5 、 -> 499 结构
499 结构 331 变化 -> 5 、


In [8]:
batch_size = 200
batch_size = 100
# batch_size = 256
embedding_size = 100    # Dimension of the embedding vector.
skip_window = 5    # How many words to consider left and right.
num_skips = 2*skip_window    # How many times to reuse an input to generate a label.
num_sampled = 100    # Number of negative examples to sample.
# num_sampled = 128    # Number of negative examples to sample.
vocabulary_size = len(id_to_word)

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
# valid_size = 16    # Random set of words to evaluate similarity on.
# valid_window = 100    # Only pick dev samples in the head of the distribution.
# valid_examples = np.random.choice(valid_window, valid_size, replace=False)
valid_examples = list(range(280, 291))
valid_size = len(valid_examples)

graph = tf.Graph()

with graph.as_default():

    # Input data.
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size, num_skips])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/gpu:0'):
        # Look up embeddings for inputs.
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(
                    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            print("embeddings:", embeddings)
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
            print("embed:", embed)
            # take mean of embeddings of context words for context embedding
#             embed_context = tf.reduce_mean(embed, 1)
            
    with tf.device('/gpu:0'):
        with tf.name_scope('attention'):
            attention_weights = tf.Variable(tf.random_uniform([embedding_size, embedding_size], -1.0, 1.0))
            embed_reshape = tf.reshape(embed, [-1, embedding_size])
            attention_matmul = tf.reshape(tf.matmul(embed_reshape, attention_weights), [batch_size, num_skips, embedding_size])
            
            target_embed = tf.nn.embedding_lookup(embeddings, train_labels)
            attention_matmul = tf.matmul(attention_matmul, tf.transpose(target_embed, perm=[0, 2, 1]))
            attention = tf.transpose(tf.nn.softmax(attention_matmul, dim=1), perm=[0, 2, 1])
            attention_context = tf.squeeze(tf.matmul(attention, embed))
            print("attention_weights:", attention_weights)
            print("attention:", attention)
            print("attention_context:", attention_context)
            
            # statistics attention info
            attention_mean, attention_var = tf.nn.moments(attention, axes=[-1])
            attention_mean = tf.reduce_mean(attention_mean)
            attention_var = tf.reduce_mean(attention_var)
            print("attention_mean:", attention_mean)
            print("attention_var:", attention_var)

        # Construct the variables for the NCE loss
        with tf.name_scope('weights'):
            nce_weights = tf.Variable(
                    tf.truncated_normal(
                            [vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
        with tf.name_scope('biases'):
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    # Explanation of the meaning of NCE loss:
    #     http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
    with tf.name_scope('loss'):
#         loss = tf.reduce_mean(
#             tf.nn.nce_loss(nce_weights, nce_biases, embed_context, train_labels,
#                            num_sampled, vocabulary_size))
#         print(train_labels, embed_context)
        loss = tf.reduce_mean(
                tf.nn.nce_loss(
                        weights=nce_weights,
                        biases=nce_biases,
                        labels=train_labels,
#                         inputs=embed_context,
                        inputs=attention_context,
#                         labels=embed_context,
#                         inputs=train_labels,
                        num_sampled=num_sampled,
                        num_classes=vocabulary_size))

    # Add the loss value as a scalar to summary.
    tf.summary.scalar('loss', loss)

    # Construct the SGD optimizer using a learning rate of 1.0.
    with tf.name_scope('optimizer'):
#         optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
#         optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
#         optimizer = tf.train.AdamOptimizer(1).minimize(loss)
        optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    print("valid_embeddings:", valid_embeddings)
    print("normalized_embeddings:", normalized_embeddings)
    print("similarity:", similarity)

    # Merge all summaries.
    merged = tf.summary.merge_all()

    # Add variable initializer.
    init = tf.global_variables_initializer()

    # Create a saver.
    saver = tf.train.Saver()
    saver_embed = tf.train.Saver([embeddings])

embeddings: <tf.Variable 'embeddings/Variable:0' shape=(507260, 100) dtype=float32_ref>
embed: Tensor("embeddings/embedding_lookup:0", shape=(100, 10, 100), dtype=float32, device=/device:GPU:0)
attention_weights: <tf.Variable 'attention/Variable:0' shape=(100, 100) dtype=float32_ref>
attention: Tensor("attention/transpose_3:0", shape=(100, 1, 10), dtype=float32, device=/device:GPU:0)
attention_context: Tensor("attention/Squeeze:0", shape=(100, 100), dtype=float32, device=/device:GPU:0)
attention_mean: Tensor("attention/Mean:0", shape=(), dtype=float32, device=/device:GPU:0)
attention_var: Tensor("attention/Mean_1:0", shape=(), dtype=float32, device=/device:GPU:0)
valid_embeddings: Tensor("embedding_lookup:0", shape=(11, 100), dtype=float32)
normalized_embeddings: Tensor("truediv:0", shape=(507260, 100), dtype=float32)
similarity: Tensor("MatMul:0", shape=(11, 507260), dtype=float32)


In [ ]:
num_steps = 20000010
# num_steps = 1
log_dir = "./log_003_cbow_attention_10win/"
log_dir = "./log_003_tmp/"
log_embed_dir = "./log_embeddings/"

tfconfig = tf.ConfigProto()
tfconfig.gpu_options.allow_growth = True

with tf.Session(graph=graph, config=tfconfig) as session:
    # Open a writer to write summaries.
    writer = tf.summary.FileWriter(log_dir, session.graph)

    # We must initialize all variables before we use them.
#     init.run()
#     saver = tf.train.import_meta_graph('./checkpoint_dir/MyModel-1000.meta')
    saver.restore(session, tf.train.latest_checkpoint(log_dir))
#     saver_embed.restore(session, tf.train.latest_checkpoint(log_embed_dir))
    print('Initialized')

    average_loss = 0
    avetage_attention = 0
    std_attention = 0
    start_index = 13178000
    for step in xrange(start_index, num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, skip_window=skip_window, num_skips=num_skips)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # Define metadata variable.
        run_metadata = tf.RunMetadata()

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        # Also, evaluate the merged op to get all summaries from the returned "summary" variable.
        # Feed metadata variable to session for visualizing the graph in TensorBoard.
        _, summary, loss_val, attention_val, std_val = session.run(
                [optimizer, merged, loss, attention_mean, attention_var],
                feed_dict=feed_dict,
                run_metadata=run_metadata)
        average_loss += loss_val
        avetage_attention += attention_val
        std_attention += std_val
        
#         print("embed:", embed.eval(feed_dict=feed_dict))
#         print("attention_w:", attention_w.eval())
#         print("attention_matmul:", attention_matmul.eval(feed_dict=feed_dict))
#         print("attention:", attention.eval(feed_dict=feed_dict))
#         print("attention_context:", attention_context.eval(feed_dict=feed_dict))

        # Add returned summaries to writer in each step.
        writer.add_summary(summary, step)
        # Add metadata to visualize the graph for the last run.
        if step == (num_steps - 1):
            writer.add_run_metadata(run_metadata, 'step%d' % step)

        if step % 2000 == 0:
            if step > 0 and step != start_index:
                average_loss /= 2000
                avetage_attention /= 2000
                std_attention /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0
            print('Average attention at step ', step, ': ', avetage_attention)
            print('Variance attention at step ', step, ': ', std_attention)
            avetage_attention = 0
            std_attention = 0
#             print("attention:", attention.eval(feed_dict=feed_dict)[:2])
#             print("attention:", attention.eval(feed_dict=feed_dict))

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = id_to_word[valid_examples[i]]
                top_k = 8    # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = id_to_word[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
                
            # Save the model for checkpoints.
            saver.save(session, os.path.join(log_dir, 'model.ckpt'), global_step=step)
            
        if step % 1000000 == 0 and step != start_index:
            word2vec = embeddings.eval()
            print(word2vec.shape, type(word2vec))
            np.save("result/003#cbow_attention_win10_"+str(step), word2vec)
            
    final_embeddings = normalized_embeddings.eval()

    # Write corresponding labels for the embeddings.
    with open(log_dir + '/metadata.tsv', 'w') as f:
        for i in xrange(vocabulary_size):
            f.write(id_to_word[i] + '\n')

    # Save the model for checkpoints.
    saver.save(session, os.path.join(log_dir, 'model.ckpt'), global_step=step)

    # Create a configuration for visualizing embeddings with the labels in TensorBoard.
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embeddings.name
    projector.visualize_embeddings(writer, config)

writer.close()

In [ ]:
Nearest to 音乐: 艺术, 节奏, 流行音乐, 摇滚, 舞蹈, 歌手, 戏剧, 表演,
Nearest to 地方: 机关, 地区, 区域, 各地, 地域, 中央政府, 八里, 秋川市,
Nearest to 社会: 观念, 道德, 伦理, 教育, 剥削, 民主, 提倡, 议题,
Nearest to 服务: 运营, 营运, 免费, 福利, 运输, 资讯, 业务, 购物,
Nearest to 均: 皆, 都, 仍, 亦, 虽, 几乎, 仍然, 广地,
Nearest to 型: 车型, W型, 电力机车, 韶山, 型号, 东风, 柴油机, 车,
Nearest to 学生: 教师, 同学, 师生, 毕业生, 老师, 该校, 校友, 家长,
Nearest to 今: 今属, 今日, 现, 治今, 郭于, 地处, 河北, 山西省,
Nearest to 受到: 受, 深受, 遭到, 遭受, 备受, 引起, 不受, 得到,
Nearest to 事件: 事故, 事情, 案件, 案, 骚乱, 一宗, 冲突, 大火,
Nearest to 经济: 农业, 金融, 产业, 制造业, 工业, 减排, 以诺, 平装书,

In [ ]:
wordsim_240 = None
with open("./data/240.txt") as f:
    wordsim_240 = f.readlines()
    wordsim_240 = [line.strip().split("\t") for line in wordsim_240]
wordsim_240[:5]

In [ ]:
log_dir = "./log_003_cbow_attention/"

# tfconfig = tf.ConfigProto(device_count={'gpu':0})
tfconfig = tf.ConfigProto()
tfconfig.gpu_options.allow_growth = True
# tfconfig.device_count = {'GPU': 0}

with tf.Session(graph=graph, config=tfconfig) as session:
    saver.restore(session, tf.train.latest_checkpoint(log_dir))
    print('Initialized')
    context_lt = []
    for item1, item2, score in wordsim_240:
        if item1 not in word_to_id:
            print(item1, item2)
            continue
        if item2 not in word_to_id:
            print(item2)
            continue
        testitem1 = word_to_id[item1]
        testitem2 = word_to_id[item2]
        sim = similarity_smi_test.eval({test_dataset:[testitem1, testitem2]})
        context_lt.append([item1, item2, sim[0][0]*10, float(score)])
        print()
        print(item1, item2, sim[0][0]*10, score)
        index, value = session.run([similarity_test_top_k_index, similarity_test_top_k_value], feed_dict={test_dataset:[testitem1]})
        for i, v in zip(index[0], value[0]):
            print(id_to_word[i], v, end=",")
        print()
        index, value = session.run([similarity_test_top_k_index, similarity_test_top_k_value], feed_dict={test_dataset:[testitem2]})
        for i, v in zip(index[0], value[0]):
            print(id_to_word[i], v, end=",")
        print()

In [ ]:
import pandas as pd
context_pd = pd.DataFrame(context_lt, columns=['a', 'b', 'pscore', 'score'])
# context_pd = context_pd[['pscore', 'score']]
context_pd['pscore'] = [float(line) for line in context_pd['pscore'].values]
context_pd['score'] = [float(line) for line in context_pd['score'].values]
context_pd.corr("spearman")

In [ ]:
context_pd